<a href="https://colab.research.google.com/github/JoaoVitorSampaio/OperationalResearch_UFPB/blob/main/Pr%C3%A1tica_PL_MixProducao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula prática: Mix de Produção
<sup>Adaptado dos exercícios 2.3 e 2.5 do livro `Pesquisa Operacional` de `Arenales, Armentano, Morabito e Yanasse`.</sup>

## Exercício 1

### Descrição do problema
Uma fundição tem de produzir 10 toneladas de um tipo de liga metálica X e, para isso, tem disponível: lingotes de ferro, grafite e sucata. Dois componentes são relevantes para a liga: carbono e silício. As tabelas a seguir fornecem a fração, em termos percentuais, desses elementos nos ingredientes disponíveis, seus custos unitários, bem como a composição da liga (isto é, porcentagens mínima e máxima de cada componente da liga).

Frações dos elementos (%) nos ingredientes e custo dos ingredientes (R$/ton):

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Carbono | 0.5 | 90 | 9 |
| Silício | 14 | - | 27 |
| Custo | 90 | 180 | 25 |

Frações (%) mínima e máxima dos componentes na liga:

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.0 | 9.5 |
|Silício | 19 | 20 |


Escreva um modelo de otimização linear para determinar as quantidades dos ingredientes para compor a liga metálica, de modo que as especificações técnicas sejam satisfeitas e o custo seja mínimo.

### Resolução

In [ ]:
# instalação e importação do pacote mip
!pip install mip

from mip import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 6.5 MB/s 


Carrega Dados

In [ ]:
# composição de cada ingrediente
a = {
    'l': {'c': 0.005, 's': 0.14},
    'g': {'c': 0.9,   's': 0.0},
    's': {'c': 0.09,  's': 0.27},
}

# custo
c = {'l': 90, 'g': 180, 's': 25}

# composições mínimas e máximas dos componentes
n = {'c': 0.0, 's': 0.19}  # min
m = {'c': 0.095, 's': 0.2} # max

# quantidade desejada da liga
Q = 10

Cria modelo

$x_i$: quantidade, em toneladas, do ingrediente $i$ usado na liga

$$\min c_l x_l + c_g x_g + c_s x_s$$

Sujeito a:

$$x_l + x_g + x_s = Q$$

$$n_c Q \leq a_{lc} x_l + a_{gc} x_g + a_{sc} x_s \leq m_c Q$$
$$n_s Q \leq a_{ls} x_l + a_{gs} x_g + a_{ss} x_s \leq m_s Q$$
$$x_l, x_g, x_s \geq 0$$

In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC)

# criação/adição da variável no modelo
x = {i: model.add_var(var_type=CONTINUOUS, name=f'x_{i}', lb=0.0) for i in  ['l', 'g', 's']}

# função objetivo
model.objective = c['l']*x['l'] + c['g']*x['g'] + c['s']*x['s']

# a soma dos ingredientes usados deve ser igual a Q
model += x['l'] + x['g'] + x['s'] == Q

# restrição na quantidade de Carbono na liga
carbono = a['l']['c']*x['l'] + a['g']['c']*x['g'] + a['s']['c']*x['s']
model += n['c']*Q <= carbono
model += carbono <= m['c']*Q

# restrição na quantidade de Silicio na liga
silicio = a['l']['s']*x['l'] + a['g']['s']*x['g'] + a['s']['s']*x['s']
model += n['s']*Q <= silicio
model += silicio <= m['s']*Q

model.write("model.lp") # salva modelo em arquivo
with open("model.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_l + 180 x_g + 25 x_s
Subject To
constr(0):  x_l + x_g + x_s = 10
constr(1):  0.00500 x_l + 0.90000 x_g + 0.09000 x_s >= -0
constr(2):  0.00500 x_l + 0.90000 x_g + 0.09000 x_s <= 0.95000
constr(3):  0.14000 x_l + 0.27000 x_s >= 1.90000
constr(4):  0.14000 x_l + 0.27000 x_s <= 2
Bounds
End



Executa

In [ ]:
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")

solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 600.00

Solution:
x_l = 5.38
x_g = 0.00
x_s = 4.62


## Exercício 2

Agora considere que os ingredientes tem o estoque limitado, de acordo com a tabela abaixo.

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Estoque (ton) | 5 | 5 | 12 |

Como o modelo pode ser modificado para atender a esse requisito?

### Código

Carrega Dados

In [ ]:
# estoque
e = {'l': 5, 'g': 5, 's': 12}

Cria modelo

Adicionamos as seguintes restrições ao modelo anterior:

$$x_l \leq e_l$$
$$x_g \leq e_g$$
$$x_s \leq e_s$$

In [ ]:
model += x['l'] <= e['l']
model += x['g'] <= e['g']
model += x['s'] <= e['s']

model.write("modelo2.lp") # salva modelo em arquivo
with open("modelo2.lp") as f: # Lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_l + 180 x_g + 25 x_s
Subject To
constr(0):  x_l + x_g + x_s = 10
constr(1):  0.00500 x_l + 0.90000 x_g + 0.09000 x_s >= -0
constr(2):  0.00500 x_l + 0.90000 x_g + 0.09000 x_s <= 0.95000
constr(3):  0.14000 x_l + 0.27000 x_s >= 1.90000
constr(4):  0.14000 x_l + 0.27000 x_s <= 2
constr(5):  x_l <= 5
constr(6):  x_g <= 5
constr(7):  x_s <= 12
Bounds
End



Executa

In [ ]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 603.70

Solution:
x_l = 5.00
x_g = 0.19
x_s = 4.81


## Exercício 3

Mantendo a restrição de estoque do Exercicio 2 suponha agora que duas ligas metálicas devem ser preparadas e os mesmos ingredientes são utilizados em ambas. A liga especificada no Exercício 1 é referida como liga 1 e devem ser produzidas 10 toneladas desta liga. Da outra liga, referida como liga 2, devem ser produzidas 6 toneladas e suas composições mínima e máxima são dadas na tabela abaixo.

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.00 | 40 |
|Silício | 12 | 19 |


### Código

Carrega dados

In [ ]:
# composições mínimas e máximas dos componentes
n = [{'c': 0.0, 's': 0.19}, {'c': 0.0, 's': 0.12}]
m = [{'c': 0.095, 's': 0.2}, {'c': 0.4, 's': 0.19}]

# quantidade desejada da liga
Q = [10, 6]

Cria modelo

$x_{li}$: quantidade, em toneladas, do ingrediente $i$ usado na liga $l$

$$\min ((c_l x_{0l} + c_g x_{0g} + c_s x_{0s}) + (c_l x_{1l} + c_g x_{1g} + c_s x_{1s}))$$

Sujeito a:

$$x_{0l} + x_{0g} + x_{0s} = Q_0$$
$$x_{1l} + x_{1g} + x_{1s} = Q_1$$

$$n_{0c} Q_0 \leq a_{lc} x_{0l} + a_{gc} x_{0g} + a_{sc} x_{0s} \leq m_{0c} Q_0$$
$$n_{0s} Q_0 \leq a_{ls} x_{0l} + a_{gs} x_{0g} + a_{ss} x_{0s} \leq m_{0s} Q_0$$

$$n_{1c} Q_1 \leq a_{lc} x_{1l} + a_{gc} x_{1g} + a_{sc} x_{1s} \leq m_{1c} Q_1$$
$$n_{1s} Q_1 \leq a_{ls} x_{1l} + a_{gs} x_{1g} + a_{ss} x_{1s} \leq m_{1s} Q_1$$


$$x_{0l} + x_{1l} \leq e_l$$
$$x_{0g} + x_{1g} \leq e_g$$
$$x_{0s} + x_{1s} \leq e_s$$

$$x_{0l}, x_{0g}, x_{0s}, x_{1l}, x_{1g}, x_{1s} \geq 0$$

In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC)

# criação/adição da variável no modelo
x = [{i: model.add_var(name=f'x_{l}_{i}') for i in ['l', 'g', 's']} for l in range(2)]

# função objetivo
model.objective = c['l']*x[0]['l'] + c['g']*x[0]['g'] + c['s']*x[0]['s'] + c['l']*x[1]['l'] + c['g']*x[1]['g'] + c['s']*x[1]['s']

# a soma dos ingredientes usados na liga 0 deve ser igual a Q0
model += x[0]['l'] + x[0]['g'] + x[0]['s'] == Q[0]

# a soma dos ingredientes usados na liga 1 deve ser igual a Q1
model += x[1]['l'] + x[1]['g'] + x[1]['s'] == Q[1]

# restrição na quantidade de Carbono na liga 0
carbono = a['l']['c']*x[0]['l'] + a['g']['c']*x[0]['g'] + a['s']['c']*x[0]['s']
model += n[0]['c']*Q[0] <= carbono
model += carbono <= m[0]['c']*Q[0]

# restrição na quantidade de Silicio na liga 0
silicio = a['l']['s']*x[0]['l'] + a['g']['s']*x[0]['g'] + a['s']['s']*x[0]['s']
model += n[0]['s']*Q[0] <= silicio
model += silicio <= m[0]['s']*Q[0]

# restrição na quantidade de Carbono na liga 1
carbono = a['l']['c']*x[1]['l'] + a['g']['c']*x[1]['g'] + a['s']['c']*x[1]['s']
model += n[1]['c']*Q[1] <= carbono
model += carbono <= m[1]['c']*Q[1]

# restrição na quantidade de Silicio na liga 1
silicio = a['l']['s']*x[1]['l'] + a['g']['s']*x[1]['g'] + a['s']['s']*x[1]['s']
model += n[1]['s']*Q[1] <= silicio
model += silicio <= m[1]['s']*Q[1]

# restrições de estoque
model += x[0]['l'] + x[1]['l'] <= e['l']
model += x[0]['g'] + x[1]['g'] <= e['g']
model += x[0]['s'] + x[1]['s'] <= e['s']

model.write("modelo3.lp") # salva modelo em arquivo
with open("modelo3.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_0_l + 180 x_0_g + 25 x_0_s + 90 x_1_l + 180 x_1_g + 25 x_1_s
Subject To
constr(0):  x_0_l + x_0_g + x_0_s = 10
constr(1):  x_1_l + x_1_g + x_1_s = 6
constr(2):  0.00500 x_0_l + 0.90000 x_0_g + 0.09000 x_0_s >= -0
constr(3):  0.00500 x_0_l + 0.90000 x_0_g + 0.09000 x_0_s <= 0.95000
constr(4):  0.14000 x_0_l + 0.27000 x_0_s >= 1.90000
constr(5):  0.14000 x_0_l + 0.27000 x_0_s <= 2
constr(6):  0.00500 x_1_l + 0.90000 x_1_g + 0.09000 x_1_s >= -0
constr(7):  0.00500 x_1_l + 0.90000 x_1_g + 0.09000 x_1_s <= 2.40000
constr(8):  0.14000 x_1_l + 0.27000 x_1_s >= 0.72000
constr(9):  0.14000 x_1_l + 0.27000 x_1_s <= 1.14000
constr(10):  x_0_l + x_1_l <= 5
constr(11):  x_0_g + x_1_g <= 5
constr(12):  x_0_s + x_1_s <= 12
Bounds
End



Executa

In [ ]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 1029.26

Solution:
x_0_l = 4.32
x_0_g = 0.51
x_0_s = 5.17
x_1_l = 0.68
x_1_g = 1.45
x_1_s = 3.87
